## Game Dev
##### tkinter is a standard Python library for GUI development.
##### We will create a 3x3 grid of buttons, each representing a cell on the Tic Tac Toe board.
##### The game's main window will contain these buttons, along with controls for selecting the game mode.

In [ ]:
import tkinter as tk
from tkinter import messagebox
import random
import os  # Import the os module for exiting once game ends

# Initialize the main window
root = tk.Tk()
root.title("Tic Tac Toe")

# Set up the game board
board = [["" for _ in range(3)] for _ in range(3)]
buttons = [[None for _ in range(3)] for _ in range(3)]
current_player = "X"
game_mode = "Two Players"  # Default game mode

def choose_mode():
    """Prompt the user to choose a game mode before starting the game."""
    global game_mode
    mode_window = tk.Toplevel(root)
    mode_window.title("Choose Game Mode")
    mode_window.geometry("300x150")
    mode_window.transient(root) # Makes the pop-up window appear on top of the main window. 
    mode_window.grab_set() # to ensure main window can't interact once modal pop up window is open.

    tk.Label(mode_window, text="Select Game Mode:", font=("Arial", 14)).pack(pady=10)
    
    tk.Button(mode_window, text="Two Players", font=("Arial", 12), command=lambda: set_mode("Two Players", mode_window)).pack(pady=5)
    tk.Button(mode_window, text="One Player vs AI", font=("Arial", 12), command=lambda: set_mode("One Player vs AI", mode_window)).pack(pady=5)

def set_mode(mode, window=None):
    """Set the game mode and close the mode selection window."""
    global game_mode
    game_mode = mode
    if window:
        window.destroy()
    initialize_board()

def initialize_board():
    """Function to initialize the game board for a new game."""
    global board, current_player
    current_player = "X"
    for i in range(3):
        for j in range(3):
            board[i][j] = ""
            buttons[i][j].config(text="", state="normal")
    root.update_idletasks()

def check_winner():
    """Check if there's a winner or if the game is a draw."""
    # Check rows and columns
    for i in range(3):
        if board[i][0] == board[i][1] == board[i][2] != "":
            return board[i][0]
        if board[0][i] == board[1][i] == board[2][i] != "":
            return board[0][i]
    
    # Check diagonals
    if board[0][0] == board[1][1] == board[2][2] != "":
        return board[0][0]
    if board[0][2] == board[1][1] == board[2][0] != "":
        return board[0][2]

    # Check for a draw
    if all(board[i][j] != "" for i in range(3) for j in range(3)):
        return "Draw"

    return None

def ai_move():
    """Make a random move for the AI."""
    global current_player
    empty_cells = [(i, j) for i in range(3) for j in range(3) if board[i][j] == ""]
    if empty_cells:
        row, col = random.choice(empty_cells)
        board[row][col] = current_player
        buttons[row][col].config(text=current_player)
        winner = check_winner()
        if winner:
            end_game(winner)
        else:
            current_player = "X"  # Switch back to player

def handle_click(row, col):
    """Handle a button click."""
    global current_player
    if board[row][col] == "":
        board[row][col] = current_player
        buttons[row][col].config(text=current_player)

        # Check for a winner or draw
        winner = check_winner()
        if winner:
            end_game(winner)
        else:
            if game_mode == "Two Players":
                # Switch player
                current_player = "O" if current_player == "X" else "X"
            elif game_mode == "One Player vs AI" and current_player == "X":
                # AI takes the turn
                current_player = "O"
                ai_move()

def end_game(winner):
    """Display the game result and ask if the user wants to continue."""
    if winner == "Draw":
        messagebox.showinfo("Game Over", "It's a draw!")
    else:
        messagebox.showinfo("Game Over", f"{winner} wins!")
    
    # Ask if the user wants to continue
    if messagebox.askyesno("Continue?", "Would you like to play again?"):
        choose_mode()  # Choose the mode again
    else:
        try:
            root.quit()     # Exit the main loop
            root.update()   # Process any remaining events
            root.destroy()  # Destroy the main window
        except:
            pass
        finally:
            os._exit(0)  # Forcefully exit the program

# Set up the buttons in the grid
for i in range(3):
    for j in range(3):
        buttons[i][j] = tk.Button(root, text="", font=("Arial", 24), width=5, height=2, borderwidth=2,
                                  command=lambda row=i, col=j: handle_click(row, col))
        buttons[i][j].grid(row=i, column=j, padx=2, pady=2)  # Minimal padding between buttons

# Start the game with the mode selection
choose_mode()
root.mainloop()